### Importante!

Este notebook funcionou com uma versão 13.3 LTS Runtime!

Não há preocupações quanto ao tamanho do cluster (memória e núcleos).

Importante: não use clusters de usuário único (escolha opções compartilhadas) para lidar com dados da Federação Lakehouse.

Insira `spark.databricks.delta.retentionDurationCheck.enabled false` durante a criação das configurações do cluster para poder usar o VACUUM corretamente neste notebook.

O objetivo deste notebook é apenas mostrar a versão dos comandos SQL para Python, **use sempre a versão SQL como referência**, pois foi a usada durante o curso SQL do Databricks.

In [0]:
%sql
CREATE FOREIGN CATALOG databricks_postgres USING CONNECTION postgres_conn 
OPTIONS (database 'lakehouse_federation')

In [0]:
display(spark.read.table("databricks_postgres.public.remuneracao_day2"))

In [0]:
spark.sql("CREATE CATALOG remuneracao;")
spark.sql("USE CATALOG remuneracao;")
spark.sql("CREATE SCHEMA explorer;")
spark.sql("USE explorer;")

In [0]:
spark.sql(
"CREATE TABLE remuneracao.explorer.remuneracao_day2" + \
"COMMENT 'Remuneracao Monthly'" + \
"AS SELECT" + \
"      *" + \
"FROM " + \
"  databricks_postgres.public.remuneracao_day2;"
)

In [0]:
display(spark.sql(
"SELECT " + \
"    cargo, COUNT(cpf)" + \
"FROM" + \
"    remuneracao.explorer.remuneracao_day2" + \
"WHERE" + \
"    cargo IS NOT null" + \
"GROUP BY cargo" + \
"ORDER BY 2 DESC" + \
"LIMIT 15"
))

### Excluindo dados

In [0]:
df_remuneracao_day2 = spark.read.table("remuneracao.explorer.remuneracao_day2")
df_remuneracao_day2.delete("CARGO = 'CABO'")

### Atualizando dados

In [0]:
df_remuneracao_day2.update(condition = "cargo = 'TERCEIRO SARGENTO'",  \
                           set = { "liquido: '41.650,92'"} )

In [0]:
df_remuneracao_day2.select('cpf', 'cargo', 'liquido')  \
    .where("cargo = 'TERCEIRO SARGENTO'")  \
    .limit(5)

### Inserindo Dados

In [0]:
df_remuneracao_day2.createOrReplaceTemporaryView("remuneracao_day2")
spark.sql(
"INSERT INTO remuneracao_day2 VALUES" +  \
"('JONAS FULANO', '***123456**', 'NARNIA', 'SABATICO', 'APRENDIZ', 'ATIVO', 2, 2022, 62, '104151', ',00', 'NENHUM', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', '500000,00', '378000,00')," +  \
"('NEYMAR', '***456789**', 'BRASIL', 'FUTEBOL', 'ATLETA', 'ATIVO', 2, 2022, 62, '1051621', ',00', 'NENHUM', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', '500000,00', '378000,00')"
)

In [0]:
display(spark.sql(
"SELECT *" + \
"FROM remuneracao_day2" + \
"WHERE" + \
"  cargo IN ('SABATICO', 'FUTEBOL')"
))

In [0]:
df_remuneracao_day2 = spark.sql("SELECT * FROM remuneracao_day2")

## Mesclando dados com atualizações!

In [0]:
df_remuneracao_day2.clone('dbfs://hive_to_uc_sync/remuneracao_day2_deep_clone', isShallow = False)
remuneracao_day2 = spark.read.load('dbfs://hive_to_uc_sync/remuneracao_day2_deep_clone')
remuneracao_day2.write.saveAsTable("remuneracao.explorer.remuneracao_deep_clone")

In [0]:
#df_remuneracao_deep_clone = spark.read.load('dbfs://hive_to_uc_sync/remuneracao_day2_deep_clone')
#df_remuneracao_deep_clone.write.mode("overwrite").option("truncate", "true").save()
spark.sql("TRUNCATE TABLE remuneracao.explorer.remuneracao_deep_clone")

In [0]:
#df_remuneracao_deep_clone.count()
display(spark.sql("SELECT COUNT(*) FROM remuneracao.explorer.remuneracao_deep_clone"))

In [0]:
#df_remuneracao_deep_clone.history()
display(spark.sql("DESCRIBE HISTORY remuneracao.explorer.remuneracao_deep_clone"))

In [0]:
spark.sql("ALTER TABLE remuneracao.explorer.remuneracao_deep_clone" + \
"RENAME TO remuneracao.explorer.remuneracao_updates;")

In [0]:
display(spark.sql("DESCRIBE HISTORY remuneracao.explorer.remuneracao_updates"))

In [0]:
spark.sql("DESCRIBE DETAIL remuneracao.explorer.remuneracao_updates;")

In [0]:
spark.sql("INSERT INTO remuneracao.explorer.remuneracao_updates VALUES" + \
"('JONAS FULANO', '***123456**', 'NARNIA', 'SABATICO', 'APRENDIZ', 'ATIVO', 2, 2022, 62, '104151', ',00', 'NENHUM', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00')," + \
"('NEYMAR', '***456789**', 'COMUNICACOES', 'MARKETING', 'INFLUENCER', 'ATIVO', 2, 2022, 215, '1426', ',00', 'ESPORTE', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', '690000,00', '200000,00')," + \
"('SILVIO SANTOS', '***519612**', 'COMUNICACOES', 'TELEVISAO', 'APRESENTADOR', 'INATIVO', 2, 2022, 627, '62473', ',00', 'NENHUM', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', '100000,00', '99999,00')," + \
"('AYRTON SENNA', '***696262**', 'ESPORTES', 'CORRIDA', 'ATLETA', 'INATIVO', 2, 2022, 682, '52677', ',00', 'NENHUM', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', ',00', '259416,00', '150161,00');"

In [0]:
spark.sql(
"MERGE INTO remuneracao.explorer.remuneracao_day2 AS re" + \
"USING remuneracao.explorer.remuneracao_updates AS up" + \
"ON re.nome = up.nome AND re.cpf = up.cpf AND re.mes = up.mes" + \
"--WHEN MATCHED THEN" + \
"--  DELETE" + \
"WHEN MATCHED AND up.funcao != "INFLUENCER" THEN" + \
"  UPDATE SET *" + \
"WHEN NOT MATCHED THEN" + \
"  INSERT *"
)

In [0]:
display(df_remuneracao_day2.history())

In [0]:
#names = ['JONAS FULANO', 'NEYMAR', 'SILVIO SANTOS', 'AYRTON SENNA']
#display(df_remuneracao_day2.filter(df_remuneracao_day2.nome.isin(names)))
display(df_remuneracao_day2.where("nome IN ('JONAS FULANO', 'NEYMAR', 'SILVIO SANTOS', 'AYRTON SENNA')"))

In [0]:
display(spark.sql(
"SELECT * FROM remuneracao.explorer.remuneracao_day2 VERSION AS OF 4" + \
"EXCEPT ALL" + \
"SELECT * FROM" + \
"remuneracao.explorer.remuneracao_day2 VERSION AS OF 2"))

In [0]:
df_remuneracao_day2.restoreToVersion(0)

In [0]:
display(df_remuneracao_day2.count(0))

In [0]:
#names = ['JONAS FULANO', 'NEYMAR', 'SILVIO SANTOS', 'AYRTON SENNA']
#display(df_remuneracao_day2.filter(df_remuneracao_day2.nome.isin(names)))
display(df_remuneracao_day2.where("nome IN ('JONAS FULANO', 'NEYMAR', 'SILVIO SANTOS', 'AYRTON SENNA')"))

# LEMBRAR:
#### Para habilitar uma duração fora do mínimo permitido (168h / 1semana) você deve definir:
`spark.databricks.delta.retentionDurationCheck.enabled falso`
como configurações de parâmetros em seu cluster ou modificando as propriedades da tabela Delta, como:
`tblProps = {`
`"delta.deletedFileRetentionDuration": "500 dias"`
`"delta.logRetentionDuration": "90 dias",`
`"databricks.delta.retentionDurationCheck.enabled": "falso"`
`}`

In [0]:
df_remuneracao_day2.vacuum(0);